In [ ]:
from phonecodes import phonecodes
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import textgrid
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
import jiwer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import soundfile as sf
import pickle
from scipy import stats
_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
'''processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")'''

# loading Hubert model
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
processor_H = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model_H = AutoModelForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

In [ ]:
predicted_df=pd.read_csv("predicted_study1_TS_max1.csv")
predicted_df= predicted_df.rename(columns={'TrainingTalkerID1': 'TrainingTalkerID'})
aggregated_data=predicted_df


aggregated_data['TestTalkerID']-=2
test_talkers = aggregated_data['TestTalkerID'].unique()
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

conditions = aggregated_data['Condition2'].unique()
exposure_talkers = aggregated_data['TrainingTalkerID'].unique()
talker_conditon=['Control', 'Multi-talker', 'Single talker', 'Talker-specific']
color_palette=["gray","limegreen","blue","red"]
shape_markers =  ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h', 'x']#['o', 's', '^', 'D', 'P', '*']
test_talkerID=["CMN_M_032","CMN_M_043","CMN_M_035","CMN_M_037"]
condition_colors = {condition: color_palette[i] for i, condition in enumerate(conditions)}
talker_shapes = {talker: shape_markers[i % len(shape_markers)] for i, talker in enumerate(exposure_talkers)}

def apply_quantile_and_qcut(group):
    group['similarity_bins'] = pd.qcut(group['similarity'], q=10, duplicates='drop')
    return group

def calculate_metrics(group):
    mean_similarity = group['similarity'].mean()
    weights=group['numCorrect']+group['numIncorrect']
    #print(group)
    weighted_accuracy = np.average(group['IsCorrect'], weights=weights)

    ci_lower, ci_upper = stats.t.interval(0.95, len(group['IsCorrect']) - 1,
                                           loc=weighted_accuracy,
                                           scale=stats.sem(group['IsCorrect']))
    return pd.Series({
        'mean_similarity': mean_similarity,
        'mean_accuracy': weighted_accuracy,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'response_count': group['numCorrect'].sum() 
    }, index=['mean_similarity', 'mean_accuracy', 'ci_lower', 'ci_upper', 'response_count'])
def compute_ci(group):
    mean = group.mean()
    std_dev = group.std()
    n = len(group)
    ci = stats.norm.interval(0.95, loc=mean, scale=std_dev/np.sqrt(n))
    return pd.Series([ci[0], ci[1]], index=['ci_low', 'ci_high'])
aggregated_data = aggregated_data[aggregated_data['Condition2'] != 'Talker-specific']

for ax, test_talker in zip(axes, test_talkers):
    TS=predicted_df[predicted_df['TestTalkerID'] == test_talker]
    TS=TS[TS['Condition2'] == 'Talker-specific']
    TSgrouped = TS.groupby('TrainingTalkerID').agg(
        similarity=('similarity','median'),
        mean_isCorrect=('IsCorrect', 'mean'),
        sum_numCorrect=('numCorrect', 'sum'),
        ci_low=('IsCorrect', lambda x: np.mean(x) - stats.norm.ppf(0.975) * np.std(x) / np.sqrt(len(x))),
        ci_high=('IsCorrect', lambda x: np.mean(x) + stats.norm.ppf(0.975) * np.std(x) / np.sqrt(len(x)))
    ).reset_index()

    ax.scatter(
        x=TSgrouped['similarity'],
        y=TSgrouped['mean_isCorrect'],
        color=condition_colors[condition],
        s=TSgrouped['sum_numCorrect']/10
    )
    ax.errorbar(
        TSgrouped['similarity'],                           
        TSgrouped['mean_isCorrect'],                            
        yerr=[TSgrouped['mean_isCorrect'] - TSgrouped['ci_low'],  
            TSgrouped['ci_high'] - TSgrouped['mean_isCorrect']],
        fmt='none',                                          
        color=condition_colors[condition],  
        alpha=0.3                                   
        #capsize=5,                                           
        #label='95% CI'
    )
    
    #continue
    if aggregated_data['TestTalkerID'].all() != 'Talker-specific':
        data = aggregated_data[aggregated_data['TestTalkerID'] == test_talker]
        #data = apply_quantile_and_qcut(data)
        #data['similarity_bins']=data['similarity_bins'].apply(lambda x: (x.left + x.right) / 2)
        for condition in conditions:
        #print(condition)
            if condition!='Talker-specific':
                condition_data = data[data['Condition2'] == condition]
                for talker in set(condition_data["TrainingTalkerID"]):
                    talkerdata= condition_data[condition_data["TrainingTalkerID"]==talker]
                    talkerdata = apply_quantile_and_qcut(talkerdata)
                    talkerdata['similarity_bins']=talkerdata['similarity_bins'].apply(lambda x: (x.left + x.right) / 2)
                    '''grouped = talkerdata.groupby('similarity_bins').agg(
                        mean_isCorrect=('IsCorrect', 'mean'),
                        sum_numCorrect=('numCorrect', 'sum'),
                        ci_low=('IsCorrect', lambda x: stats.norm.interval(0.95, loc=np.mean(x), scale=np.std(x)/np.sqrt(len(x)))[0]),
                        ci_high=('IsCorrect', lambda x: stats.norm.interval(0.95, loc=np.mean(x), scale=np.std(x)/np.sqrt(len(x)))[1])
                    ).reset_index()'''
                    grouped = talkerdata.groupby('similarity_bins').agg(
                        mean_isCorrect=('IsCorrect', 'mean'),
                        sum_numCorrect=('numCorrect', 'sum'),
                        ci_low=('IsCorrect', lambda x: np.mean(x) - stats.norm.ppf(0.975) * np.std(x) / np.sqrt(len(x))),
                        ci_high=('IsCorrect', lambda x: np.mean(x) + stats.norm.ppf(0.975) * np.std(x) / np.sqrt(len(x)))
                    ).reset_index()

                    #ci_values = talkerdata.groupby('similarity_bins')['IsCorrect'].apply(compute_ci)
                    #grouped = grouped.join(ci_values)
                    #print(grouped)
                    if condition == "Single talker":
                        ax.scatter(
                            x=grouped['similarity_bins'],
                            y=grouped['mean_isCorrect'],
                            color=condition_colors[condition],
                            s=grouped['sum_numCorrect'],
                            marker =talker_shapes[talker],
                        )
                    
                    else:
                        ax.scatter(
                                x=grouped['similarity_bins'],
                                y=grouped['mean_isCorrect'],
                                color=condition_colors[condition],
                                s=grouped['sum_numCorrect'],
                                #marker =talker_shapes[test_talkerID[0]],
                            )
                    ax.errorbar(
                        grouped['similarity_bins'],                           
                        grouped['mean_isCorrect'],                            
                        yerr=[grouped['mean_isCorrect'] - grouped['ci_low'],  
                            grouped['ci_high'] - grouped['mean_isCorrect']],
                        fmt='none',                                          
                        color=condition_colors[condition],  
                        alpha=0.3                                   
                        #capsize=5,                                           
                        #label='95% CI'
                    )

    for condition in conditions:
        condition_data = data[data['Condition2'] == condition]
        if condition == "Single talker":
            for talker in exposure_talkers:
                talker_data = condition_data[condition_data['TrainingTalkerID'] == talker]
                if len(talker_data) > 1: 

                    sns.regplot(
                        x='similarity',
                        y='Predicted_Prob',
                        data=talker_data,
                        ax=ax,
                        scatter=False,
                        color=condition_colors[condition],
                        line_kws={"linestyle": "-", "alpha": 0.3, "linewidth":2},
                        label=f'{condition} fit ({talker})',
                        #logx=True,
                        logistic=True,
                        #lowess=True,
                        ci=95,
                        #truncate=False
                    )
                    line = ax.get_lines()[-1]
                    x_fit = line.get_xdata()
                    y_fit = line.get_ydata()
                    valid_indices = ~np.isnan(x_fit) & ~np.isinf(x_fit) & ~np.isnan(y_fit) & ~np.isinf(y_fit)
                    x_fit = x_fit[valid_indices]
                    y_fit = y_fit[valid_indices]
        else:
            if len(condition_data) > 1:
                sns.regplot(
                    x='similarity',
                    y='Predicted_Prob',
                    data=condition_data,
                    ax=ax,
                    scatter=False,
                    color=condition_colors[condition],
                    line_kws={"linestyle": "-", "alpha": 0.9,"linewidth":4},
                    label=f'{condition} fit',
                    logistic=True,
                    #logx=True,
                    #lowess=True,
                    ci=95
                )
                
                line = ax.get_lines()[-1]
                x_fit = line.get_xdata()
                y_fit = line.get_ydata()
                valid_indices = ~np.isnan(x_fit) & ~np.isinf(x_fit) & ~np.isnan(y_fit) & ~np.isinf(y_fit)
                x_fit = x_fit[valid_indices]
                y_fit = y_fit[valid_indices]

        
    ax.set_title(f'Sample test talker {test_talkerID[test_talker-3]}', fontsize=14)
    ax.set_xlabel('Exposure-to-test talker-similarity of keyword', fontsize=12)
    ax.set_ylabel('Listener accuracy for keyword (mean)', fontsize=12)
    ax.set_xlim(0.2,1.05)
    ax.set_ylim(0.1,1.05)
    ax.grid(alpha=0.3)
handles, labels = ax.get_legend_handles_labels()
import matplotlib.lines as mlines

condition_handles = [mlines.Line2D([], [], color=color, linestyle="-", linewidth=2, label=f"{condition}")
    for condition, color in condition_colors.items()
]

out=condition_handles
fig.legend(
    handles=out,
    loc="upper center",
    bbox_to_anchor=(0.5, 0.96),
    title="Condition line with colors",#Talkers' shapes and 
    fontsize=9,
    ncol=6
)
plt.tight_layout(rect=[0, 0, 1, 0.89])
plt.show()